# Introduction
This notebook demonstrates how to use Yahoo Finance to download historic data and run a back test over it.

In [ ]:
// YahooHistoricFeed is part of the roboquant-extra module, so we include that also
%use roboquant(version=1.3.0-SNAPSHOT, modules=extra)


// and we import the yahoo package
import org.roboquant.yahoo.*

Welcome()

# Create the Feed

In [ ]:
val feed = YahooHistoricFeed()

// We retrieve the data for last 5 years for a number of symbols
val tf = Timeframe.past(5.years)
feed.retrieve("AAPL", "IBM", "JPM", "MSFT", "TSLA", "GOOGL", "AMZN", timeframe = tf)

In [ ]:
println(feed.assets.summary())
println(feed.timeframe)

In [ ]:
val apple = feed.assets.getBySymbol("AAPL")
PriceBarChart(feed, apple)

# Run a Strategy
We run a back test using the default EMA Crossover Strategy and use all the data in the feed.

In [ ]:
val strategy = EMAStrategy()
val roboquant = Roboquant(strategy, AccountMetric())
roboquant.run(feed)

In [ ]:
// How does the account look like at the end of the back test
val account = roboquant.broker.account
account.summary()

In [ ]:
// Plot the equity curve
val metric = roboquant.logger.getMetric("account.equity")
MetricChart(metric)

# Offline usage
It is easy to make the data just retrieved from Yahoo Finance available for offline usage. We'll be using the AvroFeed for this, which is memory efficient as well as very fast.

Please note this approach works for any feed with prices, not only a YahooHistoricFeed.

In [ ]:
// Record all the prices in the feed to a Avro file.
AvroFeed.record(feed, "/tmp/yahoo_finance.avro")

From now on you can use the AvroFeed which has the same data as the YahooHistoricFeed which we just recorded. Just to demonstrate it is the same data, we plot the Apple stock prices again but now using the AvroFeed.

In [ ]:
val feed = AvroFeed("/tmp/yahoo_finance.avro")
val apple = feed.assets.getBySymbol("AAPL")
PriceBarChart(feed, apple)